In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from sklearn import

In [8]:
li = list(range(1,25))
df = pd.DataFrame()
for x in li:
    file_path =  'raw_files/sold{}.csv'.format(x)
    frame = pd.read_csv(file_path)
    df = df.append(frame)


In [9]:
pd.set_option("display.max_rows",15, "display.max_columns", None)

In [10]:
df = df.drop([
    'SALE TYPE',
    'DAYS ON MARKET',
    'STATUS',
    'NEXT OPEN HOUSE START TIME',
    'NEXT OPEN HOUSE END TIME',
    'SOURCE',
    'FAVORITE',
    'INTERESTED',
    'HOA/MONTH'], axis = 1)
df = df.dropna()
df

,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),MLS#,LATITUDE,LONGITUDE
0,December-10-2020,Single Family Residential,1719 Glen Echo Rd,Nashville,TN,37215,1425000,4.0,5.0,"Barlow Glen, Green Hills",5055.0,10890.0,2015.0,282.0,http://www.redfin.com/TN/Nashville/1719-Glen-E...,2197831.0,36.107430,-86.805381
1,November-30-2020,Single Family Residential,4811 Granny White Pike,Nashville,TN,37220,1975000,5.0,4.5,Oak Hill,5338.0,41382.0,2020.0,370.0,http://www.redfin.com/TN/Nashville/4811-Granny...,2205803.0,36.081877,-86.805821
2,April-2-2021,Single Family Residential,1731 Glen Echo Rd,Nashville,TN,37215,1100000,4.0,5.0,Green Hills,5228.0,10018.0,2007.0,210.0,http://www.redfin.com/TN/Nashville/1731-Glen-E...,2218686.0,36.107336,-86.806303
3,February-4-2021,Single Family Residential,4027 Graybar Ct,Nashville,TN,37215,1600000,4.0,4.0,Graymont,5265.0,7405.0,2020.0,304.0,http://www.redfin.com/TN/Nashville/4027-Grayba...,2205393.0,36.113044,-86.807369
4,February-9-2021,Single Family Residential,2108 Timberwood Pl,Nashville,TN,37215,1775000,4.0,5.0,Forest Hills,5040.0,40946.0,2015.0,352.0,http://www.redfin.com/TN/Nashville/2108-Timber...,2210818.0,36.064209,-86.850463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,December-11-2020,Single Family Residential,309 Maple St,Madison,TN,37115,178000,2.0,1.0,Forest Park,728.0,7405.0,1935.0,245.0,http://www.redfin.com/TN/Madison/309-Maple-St-...,2187299.0,36.261820,-86.706953
268,December-23-2020,Single Family Residential,1015 Dew St,Nashville,TN,37206,250000,2.0,1.0,N H Hobson/Caldwell,720.0,7405.0,1950.0,347.0,http://www.redfin.com/TN/Nashville/1015-Dew-St...,2209933.0,36.166546,-86.751508
269,December-10-2020,Condo/Co-op,2601 Hillsboro Pike Unit K6,Nashville,TN,37212,150000,1.0,1.0,The Villager,600.0,435.0,1971.0,250.0,http://www.redfin.com/TN/Nashville/2601-Hillsb...,2208832.0,36.125434,-86.807731
270,February-25-2021,Single Family Residential,606 Village Ct,Nashville,TN,37206,257500,2.0,1.0,Edgefield Land,704.0,7405.0,1953.0,366.0,http://www.redfin.com/TN/Nashville/606-Village...,2220585.0,36.168491,-86.746254


In [11]:
df = df.rename(columns={"URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)": "URL"})

In [38]:
ml_df = df
ml_df = ml_df.drop([
    'SOLD DATE',
    'ADDRESS',
    'STATE OR PROVINCE',
    'URL',
    'MLS#',
    'LATITUDE',
    'LONGITUDE',
    'LOCATION'], axis =1)
ml_df

,PROPERTY TYPE,CITY,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET
0,Single Family Residential,Nashville,37215,1425000,4.0,5.0,5055.0,10890.0,2015.0,282.0
1,Single Family Residential,Nashville,37220,1975000,5.0,4.5,5338.0,41382.0,2020.0,370.0
2,Single Family Residential,Nashville,37215,1100000,4.0,5.0,5228.0,10018.0,2007.0,210.0
3,Single Family Residential,Nashville,37215,1600000,4.0,4.0,5265.0,7405.0,2020.0,304.0
4,Single Family Residential,Nashville,37215,1775000,4.0,5.0,5040.0,40946.0,2015.0,352.0
...,...,...,...,...,...,...,...,...,...,...
264,Single Family Residential,Madison,37115,178000,2.0,1.0,728.0,7405.0,1935.0,245.0
268,Single Family Residential,Nashville,37206,250000,2.0,1.0,720.0,7405.0,1950.0,347.0
269,Condo/Co-op,Nashville,37212,150000,1.0,1.0,600.0,435.0,1971.0,250.0
270,Single Family Residential,Nashville,37206,257500,2.0,1.0,704.0,7405.0,1953.0,366.0


In [39]:

ml_df.dtypes

PROPERTY TYPE          object
CITY                   object
ZIP OR POSTAL CODE      int64
PRICE                   int64
BEDS                  float64
BATHS                 float64
SQUARE FEET           float64
LOT SIZE              float64
YEAR BUILT            float64
$/SQUARE FEET         float64
dtype: object

In [40]:
ml_df['SQUARE FEET'].unique()

array([5055., 5338., 5228., ...,  742.,  727.,  623.])

In [41]:
convert_dict = {'ZIP OR POSTAL CODE': object,
                'BEDS': int,
                'SQUARE FEET': int,
                'LOT SIZE': int,
                'YEAR BUILT': object,
                '$/SQUARE FEET': int,
               }
  
ml_df = ml_df.astype(convert_dict)
ml_df.dtypes

PROPERTY TYPE          object
CITY                   object
ZIP OR POSTAL CODE     object
PRICE                   int64
BEDS                    int64
BATHS                 float64
SQUARE FEET             int64
LOT SIZE                int64
YEAR BUILT             object
$/SQUARE FEET           int64
dtype: object

In [42]:
object1 = ml_df.dtypes[ml_df.dtypes == "object"].index.to_list()

In [43]:
enc = OneHotEncoder(sparse=False)

encode_df = pd.DataFrame(enc.fit_transform(ml_df[object1]))
encode_df.columns = enc.get_feature_names(object1)
encode_df

,PROPERTY TYPE_Condo/Co-op,PROPERTY TYPE_Mobile/Manufactured Home,PROPERTY TYPE_Multi-Family (2-4 Unit),PROPERTY TYPE_Multi-Family (5+ Unit),PROPERTY TYPE_Single Family Residential,PROPERTY TYPE_Townhouse,CITY_Antioch,CITY_Ashland City,CITY_Brentwood,CITY_Franklin,CITY_Goodlettsville,CITY_Hermitage,CITY_Joelton,CITY_LA VERGNE,CITY_Madison,CITY_Mount Juliet,CITY_Nashville,CITY_Nolensville,CITY_Old Hickory,CITY_Pegram,CITY_Smyrna,CITY_Whites Creek,ZIP OR POSTAL CODE_37013,ZIP OR POSTAL CODE_37015,ZIP OR POSTAL CODE_37027,ZIP OR POSTAL CODE_37064,ZIP OR POSTAL CODE_37069,ZIP OR POSTAL CODE_37072,ZIP OR POSTAL CODE_37076,ZIP OR POSTAL CODE_37080,ZIP OR POSTAL CODE_37086,ZIP OR POSTAL CODE_37115,ZIP OR POSTAL CODE_37122,ZIP OR POSTAL CODE_37135,ZIP OR POSTAL CODE_37138,ZIP OR POSTAL CODE_37143,ZIP OR POSTAL CODE_37167,ZIP OR POSTAL CODE_37189,ZIP OR POSTAL CODE_37201,ZIP OR POSTAL CODE_37203,ZIP OR POSTAL CODE_37204,ZIP OR POSTAL CODE_37205,ZIP OR POSTAL CODE_37206,ZIP OR POSTAL CODE_37207,ZIP OR POSTAL CODE_37208,ZIP OR POSTAL CODE_37209,ZIP OR POSTAL CODE_37210,ZIP OR POSTAL CODE_37211,ZIP OR POSTAL CODE_37212,ZIP OR POSTAL CODE_37214,ZIP OR POSTAL CODE_37215,ZIP OR POSTAL CODE_37216,ZIP OR POSTAL CODE_37217,ZIP OR POSTAL CODE_37218,ZIP OR POSTAL CODE_37219,ZIP OR POSTAL CODE_37220,ZIP OR POSTAL CODE_37221,YEAR BUILT_1804.0,YEAR BUILT_1833.0,YEAR BUILT_1850.0,YEAR BUILT_1876.0,YEAR BUILT_1884.0,YEAR BUILT_1885.0,YEAR BUILT_1890.0,YEAR BUILT_1894.0,YEAR BUILT_1899.0,YEAR BUILT_1900.0,YEAR BUILT_1903.0,YEAR BUILT_1905.0,YEAR BUILT_1906.0,YEAR BUILT_1909.0,YEAR BUILT_1910.0,YEAR BUILT_1912.0,YEAR BUILT_1915.0,YEAR BUILT_1916.0,YEAR BUILT_1917.0,YEAR BUILT_1918.0,YEAR BUILT_1920.0,YEAR BUILT_1921.0,YEAR BUILT_1922.0,YEAR BUILT_1923.0,YEAR BUILT_1924.0,YEAR BUILT_1925.0,YEAR BUILT_1926.0,YEAR BUILT_1927.0,YEAR BUILT_1928.0,YEAR BUILT_1929.0,YEAR BUILT_1930.0,YEAR BUILT_1932.0,YEAR BUILT_1933.0,YEAR BUILT_1934.0,YEAR BUILT_1935.0,YEAR BUILT_1936.0,YEAR BUILT_1937.0,YEAR BUILT_1938.0,YEAR BUILT_1939.0,YEAR BUILT_1940.0,YEAR BUILT_1941.0,YEAR BUILT_1942.0,YEAR BUILT_1943.0,YEAR BUILT_1944.0,YEAR BUILT_1945.0,YEAR BUILT_1946.0,YEAR BUILT_1947.0,YEAR BUILT_1948.0,YEAR BUILT_1949.0,YEAR BUILT_1950.0,YEAR BUILT_1951.0,YEAR BUILT_1952.0,YEAR BUILT_1953.0,YEAR BUILT_1954.0,YEAR BUILT_1955.0,YEAR BUILT_1956.0,YEAR BUILT_1957.0,YEAR BUILT_1958.0,YEAR BUILT_1959.0,YEAR BUILT_1960.0,YEAR BUILT_1961.0,YEAR BUILT_1962.0,YEAR BUILT_1963.0,YEAR BUILT_1964.0,YEAR BUILT_1965.0,YEAR BUILT_1966.0,YEAR BUILT_1967.0,YEAR BUILT_1968.0,YEAR BUILT_1969.0,YEAR BUILT_1970.0,YEAR BUILT_1971.0,YEAR BUILT_1972.0,YEAR BUILT_1973.0,YEAR BUILT_1974.0,YEAR BUILT_1975.0,YEAR BUILT_1976.0,YEAR BUILT_1977.0,YEAR BUILT_1978.0,YEAR BUILT_1979.0,YEAR BUILT_1980.0,YEAR BUILT_1981.0,YEAR BUILT_1982.0,YEAR BUILT_1983.0,YEAR BUILT_1984.0,YEAR BUILT_1985.0,YEAR BUILT_1986.0,YEAR BUILT_1987.0,YEAR BUILT_1988.0,YEAR BUILT_1989.0,YEAR BUILT_1990.0,YEAR BUILT_1991.0,YEAR BUILT_1992.0,YEAR BUILT_1993.0,YEAR BUILT_1994.0,YEAR BUILT_1995.0,YEAR BUILT_1996.0,YEAR BUILT_1997.0,YEAR BUILT_1998.0,YEAR BUILT_1999.0,YEAR BUILT_2000.0,YEAR BUILT_2001.0,YEAR BUILT_2002.0,YEAR BUILT_2003.0,YEAR BUILT_2004.0,YEAR BUILT_2005.0,YEAR BUILT_2006.0,YEAR BUILT_2007.0,YEAR BUILT_2008.0,YEAR BUILT_2009.0,YEAR BUILT_2010.0,YEAR BUILT_2011.0,YEAR BUILT_2012.0,YEAR BUILT_2013.0,YEAR BUILT_2014.0,YEAR BUILT_2015.0,YEAR BUILT_2016.0,YEAR BUILT_2017.0,YEAR BUILT_2018.0,YEAR BUILT_2019.0,YEAR BUILT_2020.0,YEAR BUILT_2021.0
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [44]:
ml_encode_df = ml_df.merge(encode_df,left_index=True, right_index=True)
ml_encode_df = ml_encode_df.drop(object1,1)
ml_encode_df

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,$/SQUARE FEET,PROPERTY TYPE_Condo/Co-op,PROPERTY TYPE_Mobile/Manufactured Home,PROPERTY TYPE_Multi-Family (2-4 Unit),PROPERTY TYPE_Multi-Family (5+ Unit),PROPERTY TYPE_Single Family Residential,PROPERTY TYPE_Townhouse,CITY_Antioch,CITY_Ashland City,CITY_Brentwood,CITY_Franklin,CITY_Goodlettsville,CITY_Hermitage,CITY_Joelton,CITY_LA VERGNE,CITY_Madison,CITY_Mount Juliet,CITY_Nashville,CITY_Nolensville,CITY_Old Hickory,CITY_Pegram,CITY_Smyrna,CITY_Whites Creek,ZIP OR POSTAL CODE_37013,ZIP OR POSTAL CODE_37015,ZIP OR POSTAL CODE_37027,ZIP OR POSTAL CODE_37064,ZIP OR POSTAL CODE_37069,ZIP OR POSTAL CODE_37072,ZIP OR POSTAL CODE_37076,ZIP OR POSTAL CODE_37080,ZIP OR POSTAL CODE_37086,ZIP OR POSTAL CODE_37115,ZIP OR POSTAL CODE_37122,ZIP OR POSTAL CODE_37135,ZIP OR POSTAL CODE_37138,ZIP OR POSTAL CODE_37143,ZIP OR POSTAL CODE_37167,ZIP OR POSTAL CODE_37189,ZIP OR POSTAL CODE_37201,ZIP OR POSTAL CODE_37203,ZIP OR POSTAL CODE_37204,ZIP OR POSTAL CODE_37205,ZIP OR POSTAL CODE_37206,ZIP OR POSTAL CODE_37207,ZIP OR POSTAL CODE_37208,ZIP OR POSTAL CODE_37209,ZIP OR POSTAL CODE_37210,ZIP OR POSTAL CODE_37211,ZIP OR POSTAL CODE_37212,ZIP OR POSTAL CODE_37214,ZIP OR POSTAL CODE_37215,ZIP OR POSTAL CODE_37216,ZIP OR POSTAL CODE_37217,ZIP OR POSTAL CODE_37218,ZIP OR POSTAL CODE_37219,ZIP OR POSTAL CODE_37220,ZIP OR POSTAL CODE_37221,YEAR BUILT_1804.0,YEAR BUILT_1833.0,YEAR BUILT_1850.0,YEAR BUILT_1876.0,YEAR BUILT_1884.0,YEAR BUILT_1885.0,YEAR BUILT_1890.0,YEAR BUILT_1894.0,YEAR BUILT_1899.0,YEAR BUILT_1900.0,YEAR BUILT_1903.0,YEAR BUILT_1905.0,YEAR BUILT_1906.0,YEAR BUILT_1909.0,YEAR BUILT_1910.0,YEAR BUILT_1912.0,YEAR BUILT_1915.0,YEAR BUILT_1916.0,YEAR BUILT_1917.0,YEAR BUILT_1918.0,YEAR BUILT_1920.0,YEAR BUILT_1921.0,YEAR BUILT_1922.0,YEAR BUILT_1923.0,YEAR BUILT_1924.0,YEAR BUILT_1925.0,YEAR BUILT_1926.0,YEAR BUILT_1927.0,YEAR BUILT_1928.0,YEAR BUILT_1929.0,YEAR BUILT_1930.0,YEAR BUILT_1932.0,YEAR BUILT_1933.0,YEAR BUILT_1934.0,YEAR BUILT_1935.0,YEAR BUILT_1936.0,YEAR BUILT_1937.0,YEAR BUILT_1938.0,YEAR BUILT_1939.0,YEAR BUILT_1940.0,YEAR BUILT_1941.0,YEAR BUILT_1942.0,YEAR BUILT_1943.0,YEAR BUILT_1944.0,YEAR BUILT_1945.0,YEAR BUILT_1946.0,YEAR BUILT_1947.0,YEAR BUILT_1948.0,YEAR BUILT_1949.0,YEAR BUILT_1950.0,YEAR BUILT_1951.0,YEAR BUILT_1952.0,YEAR BUILT_1953.0,YEAR BUILT_1954.0,YEAR BUILT_1955.0,YEAR BUILT_1956.0,YEAR BUILT_1957.0,YEAR BUILT_1958.0,YEAR BUILT_1959.0,YEAR BUILT_1960.0,YEAR BUILT_1961.0,YEAR BUILT_1962.0,YEAR BUILT_1963.0,YEAR BUILT_1964.0,YEAR BUILT_1965.0,YEAR BUILT_1966.0,YEAR BUILT_1967.0,YEAR BUILT_1968.0,YEAR BUILT_1969.0,YEAR BUILT_1970.0,YEAR BUILT_1971.0,YEAR BUILT_1972.0,YEAR BUILT_1973.0,YEAR BUILT_1974.0,YEAR BUILT_1975.0,YEAR BUILT_1976.0,YEAR BUILT_1977.0,YEAR BUILT_1978.0,YEAR BUILT_1979.0,YEAR BUILT_1980.0,YEAR BUILT_1981.0,YEAR BUILT_1982.0,YEAR BUILT_1983.0,YEAR BUILT_1984.0,YEAR BUILT_1985.0,YEAR BUILT_1986.0,YEAR BUILT_1987.0,YEAR BUILT_1988.0,YEAR BUILT_1989.0,YEAR BUILT_1990.0,YEAR BUILT_1991.0,YEAR BUILT_1992.0,YEAR BUILT_1993.0,YEAR BUILT_1994.0,YEAR BUILT_1995.0,YEAR BUILT_1996.0,YEAR BUILT_1997.0,YEAR BUILT_1998.0,YEAR BUILT_1999.0,YEAR BUILT_2000.0,YEAR BUILT_2001.0,YEAR BUILT_2002.0,YEAR BUILT_2003.0,YEAR BUILT_2004.0,YEAR BUILT_2005.0,YEAR BUILT_2006.0,YEAR BUILT_2007.0,YEAR BUILT_2008.0,YEAR BUILT_2009.0,YEAR BUILT_2010.0,YEAR BUILT_2011.0,YEAR BUILT_2012.0,YEAR BUILT_2013.0,YEAR BUILT_2014.0,YEAR BUILT_2015.0,YEAR BUILT_2016.0,YEAR BUILT_2017.0,YEAR BUILT_2018.0,YEAR BUILT_2019.0,YEAR BUILT_2020.0,YEAR BUILT_2021.0
0,1425000,4,5.0,5055,10890,282,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [45]:
y = ml_encode_df["PRICE"].values
X = ml_encode_df.drop(["PRICE"],1).values



In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim= len(X[0]), kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
#evaluate the model
param_grid = {
#     'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#     'batch_size': [10, 20, 40, 60, 80, 100],
     'epochs': [10, 50, 100],
#     'learn_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
#     'momentum': [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
#     'init_mode': ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
#     'activation': ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'],
#     'weight_constraint': [1, 2, 3, 4, 5],
#     'dropout_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#     'neurons': [1, 5, 10, 15, 20, 25, 30]
    
}

model = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                      cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)
grid_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [60]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

regr = linear_model.LinearRegression()
model = regr.fit(X_train, y_train)
predictions = model.predict(X_test)
mean_squared_error(y_test,predictions)

25715390931.62689

In [67]:
pred = pd.DataFrame({'predictions': predictions, 'actual': y_test})
pred_e = pd.DataFrame({'e':abs(pred.predictions - pred.actual)})
num = (pred_e.mean()*pred_e.mean())
num//1406

e    5138378.0
dtype: float64

In [68]:
#Define the model - deep neural net
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=30, input_dim=number_input_features, activation="linear")
)
nn.add(
    tf.keras.layers.Dense(units=20, input_dim=number_input_features, activation="relu")
)

nn.add(
    tf.keras.layers.Dense(units=15, input_dim=number_input_features, activation="linear")
)
nn.add(
    tf.keras.layers.Dense(units=10, input_dim=number_input_features, activation="linear")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=8, kernel_initializer='normal'))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, ))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 30)                5520      
_________________________________________________________________
dense_9 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_10 (Dense)             (None, 15)                315       
_________________________________________________________________
dense_11 (Dense)             (None, 10)                160       
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 9         
Total params: 6,712
Trainable params: 6,712
Non-trainable params: 0
____________________________________________________

In [69]:
# Compile the model
nn.compile(loss='mean_squared_error', optimizer="adam", metrics=["accuracy"])

In [70]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=5000)

132/132 [==============================] - 1s 4ms/step - loss: 4221283019.5489 - accuracy: 0.0000e+00
Epoch 4864/5000
132/132 [==============================] - 0s 4ms/step - loss: 7972155711.5188 - accuracy: 0.0000e+00
Epoch 4865/5000
132/132 [==============================] - 0s 4ms/step - loss: 4109679532.2707 - accuracy: 0.0000e+00
Epoch 4866/5000
132/132 [==============================] - 0s 4ms/step - loss: 8884727734.8571 - accuracy: 0.0000e+00
Epoch 4867/5000
132/132 [==============================] - 0s 3ms/step - loss: 14237257818.4662 - accuracy: 0.0000e+00
Epoch 4868/5000
132/132 [==============================] - 0s 3ms/step - loss: 3092649641.3835 - accuracy: 0.0000e+00
Epoch 4869/5000
132/132 [==============================] - 0s 4ms/step - loss: 7332969085.1128 - accuracy: 0.0000e+00
Epoch 4870/5000
132/132 [==============================] - 0s 3ms/step - loss: 4950570950.2556 - accuracy: 0.0000e+00
Epoch 4871/5000
132/132 [==============================] - 0s 3ms/step 